In [2]:
import requests 
import json
import pandas as pd 
from tqdm import tqdm

Category List

In [6]:
url = "https://shopee.vn/api/v4/pages/get_homepage_category_list"
data = requests.get(url).content
data = json.loads(data)['data']["category_list"]

In [9]:
categoryList = pd.DataFrame(data)

In [10]:
categoryList.head()

,catid,parent_catid,name,display_name,image,unselected_image,selected_image,level,children,block_buyer_platform
0,11035567,0,Men Clothes,Thời Trang Nam,687f3967b7c2fe6a134a2c11894eea4b,fe2d27d140e1f34855c1bfe3168c1b58,9131b98c529e7ccfffc36193432aa927,1,None,None
1,11035639,0,Women Clothes,Thời Trang Nữ,75ea42f9eca124e9cb3cde744c060e4d,293ca6f400001a38f477ef1c8ebe2831,7002b396e4d22be3decacceeef762265,1,None,None
2,11036030,0,Mobile & Gadgets,Điện Thoại & Phụ Kiện,31234a27876fb89cd522d7e3db1ba5ca,94fb4fb95a8598e8d31f0853f904c15b,070098d9b3cf1a1d35be2a6ef1bb3cd0,1,None,None
3,11036194,0,"Moms, Kids & Babies",Mẹ & Bé,099edde1ab31df35bc255912bab54a5e,f91ae14aebd454762ab61136ebcec4e0,9b56c2810b51f8a017b76c3cefc84daf,1,None,None
4,11036132,0,Consumer Electronics,Thiết Bị Điện Tử,978b9e4cb61c611aaaf58664fae133c5,4a6a4d8230cf961e18636fbf91bad4b5,067bd7ff52bd0d3700d9b0af722e83c9,1,None,None


In [141]:
categoryList.to_csv('./Data/categoryList.csv',index=False)

Crawl ItemId & ShopId

In [155]:
# thay thế mathch_id thành id của các sản phẩm trong categoryList
url_search_item = 'https://shopee.vn/api/v4/search/search_items?by=relevancy&limit=60&match_id=11035567&newest={}'

In [89]:
def getItemShopId(data):
    itemId = data['item_basic']['itemid']
    shopId = data['item_basic']['shopid']
    return {
        'itemid':itemId,
        'shopid':shopId
    }

In [123]:
df = []
for x in tqdm(range(0,8001,60)):
    data = requests.get(url_search_item.format(x)).content
    data = json.loads(data)['items']
    res = list(map(getItemShopId,data))
    df += res

100%|██████████| 134/134 [01:47<00:00,  1.24it/s]


In [124]:
ItemShopId = pd.DataFrame(df)

In [138]:
ItemShopId

,itemid,shopid
0,5280175977,374432204
1,12840529091,235661990
2,9945980438,229626628
3,6272702484,360859859
4,13643774585,360859859
...,...,...
8035,4803987241,30720906
8036,3237502761,267107889
8037,230548610,691346
8038,3882735810,233118585


In [132]:
ItemShopId.values[0]

array([5280175977,  374432204])

In [154]:
url_detail_item = 'https://shopee.vn/api/v4/item/get?itemid={}&shopid={}'
df = []
for x in tqdm(range(ItemShopId.shape[0])):
    a,b = ItemShopId.values[0]
    data = requests.get(url_detail_item.format(a,b)).content
    data = json.loads(data)['data']
    df+=[data]

  2%|▏         | 200/8040 [01:21<53:01,  2.46it/s]  


KeyboardInterrupt: 

In [151]:
detail_item = pd.DataFrame(df)

In [153]:
detail_item.to_csv('./Data/detail_item.csv',index=False)